In [1]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup as bs
import requests
import re
import dateutil.parser as dparser
from bs4 import NavigableString
import time

In [2]:
%run ./customFunc.ipynb

In [3]:
url = "https://boardgamegeek.com/xmlapi/geeklist/30543"
xml_data=requests.get(url).content
soup=bs(xml_data, "xml")

In [4]:
forumID =[]
for item in soup.findAll('item'):
    objectidloc=str(item).split('objectid=')[1]
    objectid=objectidloc.split('"')[1]
    if str(item).find('Overview')<0:
        forumID.append(objectid)

    
    

In [5]:
len(forumID)


197

In [6]:
df=pd.DataFrame(columns=['Name', 'Rank', 'Rating', 'Votes', 'Owners', 'Plays', 'Date'])
for ID in forumID[:100]:
    forumURL='https://boardgamegeek.com/xmlapi/geeklist/'+ID
    xml_data2=requests.get(forumURL).content
    soup2=bs(xml_data2, "xml")
    try:
        date=extractDate(soup2)
        time.sleep(3)
        for item in soup2.findAll('item')[:50]:
            objectname=extractName(item)
            l=extractNum(item)
            df=pd.concat([df,makeRankInfo(l, objectname,date)], ignore_index=True)
    except:
           print(forumURL)
            

In [7]:
df2=pd.DataFrame(columns=['Name', 'Rank', 'Rating', 'Votes', 'Owners', 'Plays', 'Date'])
for ID in forumID[100:]:
    forumURL='https://boardgamegeek.com/xmlapi/geeklist/'+ID
    xml_data2=requests.get(forumURL).content
    soup2=bs(xml_data2, "xml")
    try:
        date=extractDate(soup2)
        time.sleep(3)
        for item in soup2.findAll('item')[:50]:
            objectname=extractName(item)
            l=extractNum(item)
            df2=pd.concat([df2,makeRankInfo(l, objectname,date)], ignore_index=True)
    except:
           print(forumURL)
            

In [8]:
df=pd.concat([df, df2], ignore_index=True)

In [9]:
df.to_csv('top50.csv', encoding='utf-8', index=False) 